**Opeyemi Adeniran**<br>
**COSC 611**<br>
**Final Project- Text Mining Analysis of Movie Dataset**<br>
**December 4, 2021**<br>


In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://apache.osuosl.org/spark/spark-3.1.3/spark-3.1.3-bin-hadoop2.7.tgz

In [3]:
!tar xf spark-3.1.3-bin-hadoop2.7.tgz

In [4]:
pip install -q findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.3-bin-hadoop2.7"

In [6]:
os.environ["SPARK_HOME"]

'/content/spark-3.1.3-bin-hadoop2.7'

In [7]:
pip install spark-nlp==2.4.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 108 kB 5.2 MB/s 


In [9]:
pip install pyspark==3.1.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.4.2
Apache Spark version:  3.1.3


In [11]:
pip install langid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 5.1 MB/s 
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941188 sha256=3d547ebb021c036a7698e5f6394f7d0ac1b9358ea6e67bde5a253d38b600baa5
  Stored in directory: /root/.cache/pip/wheels/c5/01/a4/0160c55074707b535a6757a541842817d530d8080ca943a107
Successfully built langid


In [12]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [1]:
import string

In [2]:
# 14.2. Text Preprocessing
# NOTE: Rerun from here if you get SparkContexts error. 
def check_blanks(data_str):
    is_blank = str(data_str.isspace())
    return is_blank

In [3]:
# Remove features
import re
import string
def remove_features(data_str):
    # compile regex
    url_re = re.compile('https?://(www.)?\w+\.\w+(/\w+)*/?')
    punc_re = re.compile('[%s]' % re.escape(string.punctuation))
    punc_re = re.compile('[^A-Za-z0-9]+') 
    num_re = re.compile('(\\d+)')
    mention_re = re.compile('@(\w+)')
    alpha_num_re = re.compile("^[a-z0-9_.]+$")
    # convert to lowercase
    data_str = data_str.lower()
    # remove hyperlinks
    data_str = url_re.sub(' ', data_str)
    # remove @mentions
    data_str = mention_re.sub(' ', data_str)
    # remove puncuation
    data_str = punc_re.sub(' ', data_str)
    # remove numeric 'words'
    data_str = num_re.sub(' ', data_str)
    # remove non a-z 0-9 characters and words shorter than 3 characters
    list_pos = 0
    cleaned_str = ''
    for word in data_str.split():
        if list_pos == 0:
            if alpha_num_re.match(word) and len(word) > 2:
                cleaned_str = word
            else:
                cleaned_str = ' '
        else:
            if alpha_num_re.match(word) and len(word) > 2:
                cleaned_str = cleaned_str + ' ' + word
            else:
                cleaned_str += ' '
        list_pos += 1
    return cleaned_str

In [4]:
# Removes stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords.words('english')
print(stopwords.words('english'))
#print(stops)
def remove_stops(data_str):
    # expects a string
    stopwords = nltk.corpus.stopwords.words('english')
    list_pos = 0
    cleaned_str = ''
    text = data_str.split()
    for word in text:
        if word not in stopwords:
            # rebuild cleaned_str
            if list_pos == 0:
                cleaned_str = word
            else:
                cleaned_str = cleaned_str + ' ' + word
            list_pos += 1
    return cleaned_str

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Tagging text
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')
def tag_and_remove(data_str):
    cleaned_str = ' '
    # noun tags
    nn_tags = ['NN', 'NNP', 'NNP', 'NNPS', 'NNS']
    # adjectives
    jj_tags = ['JJ', 'JJR', 'JJS']
    # verbs
    vb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    nltk_tags = nn_tags + jj_tags + vb_tags

    # break string into 'words'
    text = data_str.split()

    # tag the text and keep only those with the right tags
    tagged_text = nltk.pos_tag(text)
    for tagged_word in tagged_text:
        if tagged_word[1] in nltk_tags:
            cleaned_str += tagged_word[0] + ' '

    return cleaned_str

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [6]:
# Lemmatization 
import re
import string
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
def lemmatize(data_str):
    # expects a string
    list_pos = 0
    cleaned_str = ''
    lmtzr = WordNetLemmatizer()
    text = data_str.split()
    tagged_words = nltk.pos_tag(text)
    for word in tagged_words:
        if 'v' in word[1].lower():
            lemma = lmtzr.lemmatize(word[0], pos='v')
        else:
            lemma = lmtzr.lemmatize(word[0], pos='n')
        if list_pos == 0:
            cleaned_str = lemma
        else:
            cleaned_str = cleaned_str + ' ' + lemma
        list_pos += 1
    return cleaned_str

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
'''
# Functions provided for preprocessing - Please ignore
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
#import preproc as pp
check_lang_udf = udf(check_lang, StringType())
remove_stops_udf = udf(remove_stops, StringType())
remove_features_udf = udf(remove_features, StringType())
tag_and_remove_udf = udf(tag_and_remove, StringType())
lemmatize_udf = udf(lemmatize, StringType())
check_blanks_udf = udf(check_blanks, StringType())
'''

'\n# Functions provided for preprocessing - Please ignore\nfrom pyspark.sql.functions import udf\nfrom pyspark.sql.types import StringType\n#import preproc as pp\ncheck_lang_udf = udf(check_lang, StringType())\nremove_stops_udf = udf(remove_stops, StringType())\nremove_features_udf = udf(remove_features, StringType())\ntag_and_remove_udf = udf(tag_and_remove, StringType())\nlemmatize_udf = udf(lemmatize, StringType())\ncheck_blanks_udf = udf(check_blanks, StringType())\n'

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Execution for loading dataset
import pyspark
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

#import preproc as pp
'''
check_lang_udf = udf(check_lang, StringType())
'''
remove_stops_udf = udf(remove_stops, StringType())
remove_features_udf = udf(remove_features, StringType())
tag_and_remove_udf = udf(tag_and_remove, StringType())
lemmatize_udf = udf(lemmatize, StringType())
check_blanks_udf = udf(check_blanks, StringType())

# create Spark context with Spark configuration

sc = pyspark.SparkContext()
sqlContext = SQLContext(sc)
'''
# import
from pyspark.sql import HiveContext
sc = pyspark.SparkContext()
sqlContext = HiveContext(sc)
#spark = SparkSession.builder.appName("DataFrame").getOrCreate()
'''
# Uncomment here
# Read file into RDD

neg_rdd = sc.textFile("/content/drive/MyDrive/FINAL PROJECT/POLORITY/all_text/neg.txt")
pos_rdd = sc.textFile("/content/drive/MyDrive/FINAL PROJECT/POLORITY/all_text/pos.txt")

neg_parts_rdd = neg_rdd.map(lambda l: l.split("\t"))
pos_parts_rdd = neg_rdd.map(lambda l: l.split("\t"))
# Filter bad rows out
neg_filled_rdd = neg_parts_rdd.filter(bool)
pos_filled_rdd = pos_parts_rdd.filter(bool)
#typed_rdd = data_rdd.map(lambda p: (p[0], p[1], float(p[2])))

#Create DataFrame
neg_data_df = sqlContext.createDataFrame(neg_filled_rdd, StringType())
pos_data_df = sqlContext.createDataFrame(pos_filled_rdd, StringType())
# get the raw columns
neg_raw_cols = neg_data_df.columns
pos_raw_cols = pos_data_df.columns
#raw_cols = lines.columns

neg_data_df.printSchema()
pos_data_df.printSchema()
#lines.printSchema()
neg_data_df.na.drop(how="any").show(truncate=False)
pos_data_df.na.drop(how="any").show(truncate=False)

root
 |-- value: string (nullable = true)

root
 |-- value: string (nullable = true)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                         |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[in the past , tim burton has taken cinema by storm with the action packed batman , and the hilarious ed wood , but lately his films just don't cut it . ]                    |
|[sleepy hollow has some unique features , but the script is a terrible mess . ]                                                                                               |
|[for those who remember disn

In [10]:
# Remove stopwords from values
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords.words('english')
neg_rm_stops_df = neg_data_df.select(neg_raw_cols).withColumn("neg_stop_text", remove_stops_udf(neg_data_df["value"]))
neg_rm_stops_df.show(20)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


+--------------------+--------------------+
|               value|       neg_stop_text|
+--------------------+--------------------+
|[in the past , ti...|[in past , tim bu...|
|[sleepy hollow ha...|[sleepy hollow un...|
|[for those who re...|[for remember dis...|
|[disney's version...|[disney's version...|
|[while the storie...|[while stories si...|
|[ichabod crane ( ...|[ichabod crane ( ...|
|[the victims are ...|[the victims head...|
|[now with the hel...|[now help christi...|
|[sure the headles...|[sure headless ho...|
|[in the original ...|[in original , he...|
|[no one could sto...|[no one could sto...|
|[by the end of th...|[by end film , au...|
|[burton however c...|[burton however c...|
|[he takes no risk...|[he takes risk ev...|
|[another pathetic...|[another pathetic...|
|[like i said earl...|[like said earlie...|
|[he completely bu...|[he completely bu...|
|[thankfully there...|[thankfully facto...|
|[the acting by de...|[the acting depp ...|
|[he took the nerd...|[he took n

In [11]:
# Remove stopwords from values
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords.words('english')
pos_rm_stops_df = pos_data_df.select(pos_raw_cols).withColumn("pos_stop_text", remove_stops_udf(pos_data_df["value"]))
pos_rm_stops_df.show(20)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


+--------------------+--------------------+
|               value|       pos_stop_text|
+--------------------+--------------------+
|[in the past , ti...|[in past , tim bu...|
|[sleepy hollow ha...|[sleepy hollow un...|
|[for those who re...|[for remember dis...|
|[disney's version...|[disney's version...|
|[while the storie...|[while stories si...|
|[ichabod crane ( ...|[ichabod crane ( ...|
|[the victims are ...|[the victims head...|
|[now with the hel...|[now help christi...|
|[sure the headles...|[sure headless ho...|
|[in the original ...|[in original , he...|
|[no one could sto...|[no one could sto...|
|[by the end of th...|[by end film , au...|
|[burton however c...|[burton however c...|
|[he takes no risk...|[he takes risk ev...|
|[another pathetic...|[another pathetic...|
|[like i said earl...|[like said earlie...|
|[he completely bu...|[he completely bu...|
|[thankfully there...|[thankfully facto...|
|[the acting by de...|[the acting depp ...|
|[he took the nerd...|[he took n

In [12]:
# Remove features from values
import re
import string
neg_rm_features_df = neg_rm_stops_df.select(neg_raw_cols+["neg_stop_text"]).withColumn("neg_feat_text", remove_features_udf(neg_rm_stops_df["neg_stop_text"]))
neg_rm_features_df.show(20)

+--------------------+--------------------+--------------------+
|               value|       neg_stop_text|       neg_feat_text|
+--------------------+--------------------+--------------------+
|[in the past , ti...|[in past , tim bu...|  past tim burton...|
|[sleepy hollow ha...|[sleepy hollow un...|sleepy hollow uni...|
|[for those who re...|[for remember dis...|for remember disn...|
|[disney's version...|[disney's version...|disney  version l...|
|[while the storie...|[while stories si...|while stories sim...|
|[ichabod crane ( ...|[ichabod crane ( ...|ichabod crane joh...|
|[the victims are ...|[the victims head...|the victims headl...|
|[now with the hel...|[now help christi...|now help christin...|
|[sure the headles...|[sure headless ho...|sure headless hor...|
|[in the original ...|[in original , he...|  original headle...|
|[no one could sto...|[no one could sto...|  one could stop ...|
|[by the end of th...|[by end film , au...|  end film audien...|
|[burton however c...|[bu

In [13]:
# Remove features from values
import re
import string
pos_rm_features_df = pos_rm_stops_df.select(pos_raw_cols+["pos_stop_text"]).withColumn("pos_feat_text", remove_features_udf(pos_rm_stops_df["pos_stop_text"]))
pos_rm_features_df.show(20)

+--------------------+--------------------+--------------------+
|               value|       pos_stop_text|       pos_feat_text|
+--------------------+--------------------+--------------------+
|[in the past , ti...|[in past , tim bu...|  past tim burton...|
|[sleepy hollow ha...|[sleepy hollow un...|sleepy hollow uni...|
|[for those who re...|[for remember dis...|for remember disn...|
|[disney's version...|[disney's version...|disney  version l...|
|[while the storie...|[while stories si...|while stories sim...|
|[ichabod crane ( ...|[ichabod crane ( ...|ichabod crane joh...|
|[the victims are ...|[the victims head...|the victims headl...|
|[now with the hel...|[now help christi...|now help christin...|
|[sure the headles...|[sure headless ho...|sure headless hor...|
|[in the original ...|[in original , he...|  original headle...|
|[no one could sto...|[no one could sto...|  one could stop ...|
|[by the end of th...|[by end film , au...|  end film audien...|
|[burton however c...|[bu

In [17]:
# Tagging text from value
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')
neg_tagged_df = neg_rm_features_df.select(neg_raw_cols+["neg_feat_text"]).withColumn("neg_tagged_text", tag_and_remove_udf(neg_rm_features_df.neg_feat_text))
neg_tagged_df.show(20)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


+--------------------+--------------------+--------------------+
|               value|       neg_feat_text|     neg_tagged_text|
+--------------------+--------------------+--------------------+
|[in the past , ti...|  past tim burton...| past tim burton ...|
|[sleepy hollow ha...|sleepy hollow uni...| sleepy hollow un...|
|[for those who re...|for remember disn...| remember disney ...|
|[disney's version...|disney  version l...| disney version l...|
|[while the storie...|while stories sim...| stories similar ...|
|[ichabod crane ( ...|ichabod crane joh...| ichabod crane jo...|
|[the victims are ...|the victims headl...| victims headless...|
|[now with the hel...|now help christin...| help christina r...|
|[sure the headles...|sure headless hor...| sure headless ho...|
|[in the original ...|  original headle...| original headles...|
|[no one could sto...|  one could stop ...|       stop ichabod |
|[by the end of th...|  end film audien...| end film audienc...|
|[burton however c...|bur

In [18]:
# Tagging text from value
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')
pos_tagged_df = pos_rm_features_df.select(pos_raw_cols+["pos_feat_text"]).withColumn("pos_tagged_text", tag_and_remove_udf(pos_rm_features_df.pos_feat_text))
pos_tagged_df.show(20)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


+--------------------+--------------------+--------------------+
|               value|       pos_feat_text|     pos_tagged_text|
+--------------------+--------------------+--------------------+
|[in the past , ti...|  past tim burton...| past tim burton ...|
|[sleepy hollow ha...|sleepy hollow uni...| sleepy hollow un...|
|[for those who re...|for remember disn...| remember disney ...|
|[disney's version...|disney  version l...| disney version l...|
|[while the storie...|while stories sim...| stories similar ...|
|[ichabod crane ( ...|ichabod crane joh...| ichabod crane jo...|
|[the victims are ...|the victims headl...| victims headless...|
|[now with the hel...|now help christin...| help christina r...|
|[sure the headles...|sure headless hor...| sure headless ho...|
|[in the original ...|  original headle...| original headles...|
|[no one could sto...|  one could stop ...|       stop ichabod |
|[by the end of th...|  end film audien...| end film audienc...|
|[burton however c...|bur

In [19]:
# Lemmatization for values
import re
import string
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
neg_lemm_df = neg_tagged_df.select(neg_raw_cols+["neg_tagged_text"]).withColumn("neg_lemm_text", lemmatize_udf(neg_tagged_df["neg_tagged_text"]))
neg_lemm_df.show(20)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


+--------------------+--------------------+--------------------+
|               value|     neg_tagged_text|       neg_lemm_text|
+--------------------+--------------------+--------------------+
|[in the past , ti...| past tim burton ...|past tim burton t...|
|[sleepy hollow ha...| sleepy hollow un...|sleepy hollow uni...|
|[for those who re...| remember disney ...|remember disney f...|
|[disney's version...| disney version l...|disney version le...|
|[while the storie...| stories similar ...|story similar new...|
|[ichabod crane ( ...| ichabod crane jo...|ichabod crane joh...|
|[the victims are ...| victims headless...|victim headless t...|
|[now with the hel...| help christina r...|help christina ri...|
|[sure the headles...| sure headless ho...|sure headless hor...|
|[in the original ...| original headles...|original headless...|
|[no one could sto...|       stop ichabod |        stop ichabod|
|[by the end of th...| end film audienc...|end film audience...|
|[burton however c...| bu

In [21]:
# Lemmatization for values
import re
import string
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
pos_lemm_df = pos_tagged_df.select(pos_raw_cols+["pos_tagged_text"]).withColumn("pos_lemm_text", lemmatize_udf(pos_tagged_df["pos_tagged_text"]))
pos_lemm_df.show(20)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


+--------------------+--------------------+--------------------+
|               value|     pos_tagged_text|       pos_lemm_text|
+--------------------+--------------------+--------------------+
|[in the past , ti...| past tim burton ...|past tim burton t...|
|[sleepy hollow ha...| sleepy hollow un...|sleepy hollow uni...|
|[for those who re...| remember disney ...|remember disney f...|
|[disney's version...| disney version l...|disney version le...|
|[while the storie...| stories similar ...|story similar new...|
|[ichabod crane ( ...| ichabod crane jo...|ichabod crane joh...|
|[the victims are ...| victims headless...|victim headless t...|
|[now with the hel...| help christina r...|help christina ri...|
|[sure the headles...| sure headless ho...|sure headless hor...|
|[in the original ...| original headles...|original headless...|
|[no one could sto...|       stop ichabod |        stop ichabod|
|[by the end of th...| end film audienc...|end film audience...|
|[burton however c...| bu

In [24]:
neg_check_blanks_df = neg_lemm_df.select(neg_raw_cols+["neg_lemm_text"]).withColumn("is_blank", check_blanks_udf(neg_lemm_df["neg_lemm_text"]))
# remove blanks from lemmatized text
neg_no_blanks_df = neg_check_blanks_df.filter(neg_check_blanks_df["is_blank"] == "False")

# drop duplicates from lemmatized text
neg_dedup_df = neg_no_blanks_df.dropDuplicates(['neg_lemm_text'])

neg_dedup_df.show(20)

+--------------------+--------------------+--------+
|               value|       neg_lemm_text|is_blank|
+--------------------+--------------------+--------+
|[the acting is pr...|act bad jackie ch...|   False|
|[some action film...|action film actio...|   False|
|[the action scene...|action scene shoo...|   False|
|[now that he has ...|adoptive son burt...|   False|
|[the aforemention...|aforementioned cl...|   False|
|[alas betty has t...|ala dubious disti...|   False|
|[apparently , thi...|alien lifeform su...|   False|
|[is there anythin...|       anything good|   False|
|[and is there any...|anything spectacu...|   False|
|[apart from these...|apart factor noth...|   False|
|[the army should ...|army save time br...|   False|
|      [art ! " ) , ]|                 art|   False|
|[astaire and roge...|      astaire rogers|   False|
|[any attempt at a...|attempt anything ...|   False|
|[plus , the attem...|attempt make char...|   False|
|[its not that bad...|bad movie great h...|   

In [26]:
pos_check_blanks_df = pos_lemm_df.select(pos_raw_cols+["pos_lemm_text"]).withColumn("is_blank", check_blanks_udf(pos_lemm_df["pos_lemm_text"]))
# remove blanks from lemmatized text
pos_no_blanks_df = pos_check_blanks_df.filter(pos_check_blanks_df["is_blank"] == "False")

# drop duplicates from lemmatized text
pos_dedup_df = pos_no_blanks_df.dropDuplicates(['pos_lemm_text'])

pos_dedup_df.show(20)

+--------------------+--------------------+--------+
|               value|       pos_lemm_text|is_blank|
+--------------------+--------------------+--------+
|[the acting is pr...|act bad jackie ch...|   False|
|[some action film...|action film actio...|   False|
|[the action scene...|action scene shoo...|   False|
|[now that he has ...|adoptive son burt...|   False|
|[the aforemention...|aforementioned cl...|   False|
|[alas betty has t...|ala dubious disti...|   False|
|[apparently , thi...|alien lifeform su...|   False|
|[is there anythin...|       anything good|   False|
|[and is there any...|anything spectacu...|   False|
|[apart from these...|apart factor noth...|   False|
|[the army should ...|army save time br...|   False|
|      [art ! " ) , ]|                 art|   False|
|[astaire and roge...|      astaire rogers|   False|
|[any attempt at a...|attempt anything ...|   False|
|[plus , the attem...|attempt make char...|   False|
|[its not that bad...|bad movie great h...|   

In [27]:
# Rename final column as text for processing
pos_final_df = pos_dedup_df.selectExpr("pos_lemm_text as text")
pos_final_df.show()
pos_final_df.printSchema()

+--------------------+
|                text|
+--------------------+
|film rat director...|
|   movie originality|
|       anything good|
|             letdown|
|think joblo get l...|
|anything spectacu...|
|attempt make char...|
|yank us water pis...|
|let begin say wor...|
|fill brim wham ba...|
|hand controversia...|
|highlight movie a...|
|imgen company fun...|
|performance under...|
|fall hop joel sch...|
|kelsey grammer go...|
|response big lecture|
|try catch stupid ...|
|show bad as want ...|
|                 art|
+--------------------+
only showing top 20 rows

root
 |-- text: string (nullable = true)



In [28]:
# Rename final column as text for processing
neg_final_df = neg_dedup_df.selectExpr("neg_lemm_text as text")
neg_final_df.show()
neg_final_df.printSchema()

+--------------------+
|                text|
+--------------------+
|film rat director...|
|   movie originality|
|       anything good|
|             letdown|
|think joblo get l...|
|anything spectacu...|
|attempt make char...|
|yank us water pis...|
|let begin say wor...|
|fill brim wham ba...|
|hand controversia...|
|highlight movie a...|
|imgen company fun...|
|performance under...|
|fall hop joel sch...|
|kelsey grammer go...|
|response big lecture|
|try catch stupid ...|
|show bad as want ...|
|                 art|
+--------------------+
only showing top 20 rows

root
 |-- text: string (nullable = true)



In [29]:
from pyspark.sql.functions import monotonically_increasing_id
# Create Unique ID for text
neg_uid_df = neg_final_df.withColumn("neg_uid", monotonically_increasing_id())
neg_uid_df.show(4)

+--------------------+-------+
|                text|neg_uid|
+--------------------+-------+
|film rat director...|      0|
|   movie originality|      1|
|       anything good|      2|
|             letdown|      3|
+--------------------+-------+
only showing top 4 rows



In [30]:
from pyspark.sql.functions import monotonically_increasing_id
# Create Unique ID for text
pos_uid_df = pos_final_df.withColumn("pos_uid", monotonically_increasing_id())
pos_uid_df.show(4)

+--------------------+-------+
|                text|pos_uid|
+--------------------+-------+
|film rat director...|      0|
|   movie originality|      1|
|       anything good|      2|
|             letdown|      3|
+--------------------+-------+
only showing top 4 rows



In [44]:
# Create label with 1.0 being the constant for ML classifier
from pyspark.sql.functions import udf

@udf("double")
def const_col():
    return 1.0

neg_label_df = neg_uid_df.withColumn('label', const_col())
neg_label_df.show()

+--------------------+-------+-----+
|                text|neg_uid|label|
+--------------------+-------+-----+
|film rat director...|      0|  1.0|
|   movie originality|      1|  1.0|
|       anything good|      2|  1.0|
|             letdown|      3|  1.0|
|think joblo get l...|      4|  1.0|
|anything spectacu...|      5|  1.0|
|attempt make char...|      6|  1.0|
|yank us water pis...|      7|  1.0|
|let begin say wor...|      8|  1.0|
|fill brim wham ba...|      9|  1.0|
|hand controversia...|     10|  1.0|
|highlight movie a...|     11|  1.0|
|imgen company fun...|     12|  1.0|
|performance under...|     13|  1.0|
|fall hop joel sch...|     14|  1.0|
|kelsey grammer go...|     15|  1.0|
|response big lecture|     16|  1.0|
|try catch stupid ...|     17|  1.0|
|show bad as want ...|     18|  1.0|
|                 art|     19|  1.0|
+--------------------+-------+-----+
only showing top 20 rows



In [45]:
# Create label with 1.0 being the constant for ML classifier
from pyspark.sql.functions import udf

@udf("double")
def const_col():
    return 1.0

pos_label_df = pos_uid_df.withColumn('label', const_col())
pos_label_df.show()

+--------------------+-------+-----+
|                text|pos_uid|label|
+--------------------+-------+-----+
|film rat director...|      0|  1.0|
|   movie originality|      1|  1.0|
|       anything good|      2|  1.0|
|             letdown|      3|  1.0|
|think joblo get l...|      4|  1.0|
|anything spectacu...|      5|  1.0|
|attempt make char...|      6|  1.0|
|yank us water pis...|      7|  1.0|
|let begin say wor...|      8|  1.0|
|fill brim wham ba...|      9|  1.0|
|hand controversia...|     10|  1.0|
|highlight movie a...|     11|  1.0|
|imgen company fun...|     12|  1.0|
|performance under...|     13|  1.0|
|fall hop joel sch...|     14|  1.0|
|kelsey grammer go...|     15|  1.0|
|response big lecture|     16|  1.0|
|try catch stupid ...|     17|  1.0|
|show bad as want ...|     18|  1.0|
|                 art|     19|  1.0|
+--------------------+-------+-----+
only showing top 20 rows



In [58]:
pos_data = pos_label_df.select('pos_uid','text','label')
pos_data.show(20)

+-------+--------------------+-----+
|pos_uid|                text|label|
+-------+--------------------+-----+
|      0|film rat director...|  1.0|
|      1|   movie originality|  1.0|
|      2|       anything good|  1.0|
|      3|             letdown|  1.0|
|      4|think joblo get l...|  1.0|
|      5|anything spectacu...|  1.0|
|      6|attempt make char...|  1.0|
|      7|yank us water pis...|  1.0|
|      8|let begin say wor...|  1.0|
|      9|fill brim wham ba...|  1.0|
|     10|hand controversia...|  1.0|
|     11|highlight movie a...|  1.0|
|     12|imgen company fun...|  1.0|
|     13|performance under...|  1.0|
|     14|fall hop joel sch...|  1.0|
|     15|kelsey grammer go...|  1.0|
|     16|response big lecture|  1.0|
|     17|try catch stupid ...|  1.0|
|     18|show bad as want ...|  1.0|
|     19|                 art|  1.0|
+-------+--------------------+-----+
only showing top 20 rows



In [59]:
neg_data = neg_label_df.select('neg_uid','text','label')
neg_data.show(20)

+-------+--------------------+-----+
|neg_uid|                text|label|
+-------+--------------------+-----+
|      0|film rat director...|  1.0|
|      1|   movie originality|  1.0|
|      2|       anything good|  1.0|
|      3|             letdown|  1.0|
|      4|think joblo get l...|  1.0|
|      5|anything spectacu...|  1.0|
|      6|attempt make char...|  1.0|
|      7|yank us water pis...|  1.0|
|      8|let begin say wor...|  1.0|
|      9|fill brim wham ba...|  1.0|
|     10|hand controversia...|  1.0|
|     11|highlight movie a...|  1.0|
|     12|imgen company fun...|  1.0|
|     13|performance under...|  1.0|
|     14|fall hop joel sch...|  1.0|
|     15|kelsey grammer go...|  1.0|
|     16|response big lecture|  1.0|
|     17|try catch stupid ...|  1.0|
|     18|show bad as want ...|  1.0|
|     19|                 art|  1.0|
+-------+--------------------+-----+
only showing top 20 rows



In [60]:
# Split the data into training and test sets (40% held out for testing)
(pos_trainingData, pos_testData) = pos_data.randomSplit([0.6, 0.4])

In [61]:
# Split the data into training and test sets (40% held out for testing)
(neg_trainingData, neg_testData) = neg_data.randomSplit([0.6, 0.4])

In [62]:
# Running Naive Bayes classifier.
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# Configure an ML pipeline, which consists of tree stages: tokenizer, hashingTF, and nb.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
#data = tokenizer.transform(data)
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="rawFeatures")
labelEncoder = StringIndexer(inputCol="text", outputCol='label')
#vectorizer = CountVectorizer(inputCol= "words", outputCol="rawFeatures")
idf = IDF(minDocFreq=2000, inputCol="rawFeatures", outputCol="features")
#assembler = VectorAssembler(inputCols=["hour", "mobile", "userFeatures"],outputCol="features")
#idfModel = idf.fit(data)

#lda = LDA(k=20, seed=1, optimizer="em")

# DecisionTree Classifier
#dt = DecisionTreeClassifier()

# Linear Support Vector Classifer
#lsvc = LinearSVC((maxIter=10, regParam=0.1)

# Naive Bayes model
nb = NaiveBayes(smoothing=2.0)

# Random Forest Classifier
#rfc = RandomForestClassifier()

# Pipeline Architecture for NB
pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, nb])

# Pipeline Architecture for RFC
#pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, rfc])

# Pipeline Architecture for DT
#pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, dt])

# Pipeline Architecture for LSVC
#pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, lsvc])

PipelineModel_55a67adacadb

In [69]:
# Train model.  This also runs the indexers.
neg_model = pipeline.fit(neg_trainingData)
neg_model


PipelineModel_aa7dc5f8a8d0

In [64]:
# Train model.  This also runs the indexers.
pos_model = pipeline.fit(pos_trainingData)
pos_model


PipelineModel_0f070dbda5e2

In [68]:
pos_predictions = pos_model.transform(pos_testData)

# Select example rows to display.
pos_predictions.select("text", "label", "prediction").show(5,True)

+--------------------+-----+----------+
|                text|label|prediction|
+--------------------+-----+----------+
|think joblo get l...|  1.0|       0.0|
|anything spectacu...|  1.0|       0.0|
|yank us water pis...|  1.0|       0.0|
|fill brim wham ba...|  1.0|       0.0|
|imgen company fun...|  1.0|       0.0|
+--------------------+-----+----------+
only showing top 5 rows



In [71]:
neg_predictions = neg_model.transform(neg_testData)

# Select example rows to display.
neg_predictions.select("text", "label", "prediction").show(5,True)

+--------------------+-----+----------+
|                text|label|prediction|
+--------------------+-----+----------+
|film rat director...|  1.0|       0.0|
|       anything good|  1.0|       0.0|
|anything spectacu...|  1.0|       0.0|
|attempt make char...|  1.0|       0.0|
|let begin say wor...|  1.0|       0.0|
+--------------------+-----+----------+
only showing top 5 rows



In [72]:
from pyspark.ml.evaluation import RegressionEvaluator
neg_evaluator = RegressionEvaluator(predictionCol="prediction")
neg_evaluator.evaluate(neg_predictions)

1.0

In [73]:
from pyspark.ml.evaluation import RegressionEvaluator
pos_evaluator = RegressionEvaluator(predictionCol="prediction")
pos_evaluator.evaluate(pos_predictions)

0.9999999999999999

In [74]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
pos_evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
pos_evaluator.evaluate(pos_predictions)

0.0

In [75]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
neg_evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
neg_evaluator.evaluate(neg_predictions)

0.0

In [76]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
pos_evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")
pos_evaluator.evaluate(pos_predictions)

1.0

In [77]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
neg_evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")
neg_evaluator.evaluate(neg_predictions)

1.0

In [79]:
from pyspark.sql.types import FloatType

from textblob import TextBlob

def sentiment_analysis(text):
    return TextBlob(text).sentiment.polarity

sentiment_analysis_udf = udf(sentiment_analysis , FloatType())

In [80]:
df = neg_label_df.withColumn("sentiment_score", sentiment_analysis_udf(neg_label_df['text'] ))
df.show(20,True)

+--------------------+-------+-----+---------------+
|                text|neg_uid|label|sentiment_score|
+--------------------+-------+-----+---------------+
|film rat director...|      0|  1.0|         0.3125|
|   movie originality|      1|  1.0|            0.0|
|       anything good|      2|  1.0|            0.7|
|             letdown|      3|  1.0|            0.0|
|think joblo get l...|      4|  1.0|    -0.14107142|
|anything spectacu...|      5|  1.0|            0.6|
|attempt make char...|      6|  1.0|           0.15|
|yank us water pis...|      7|  1.0|            0.0|
|let begin say wor...|      8|  1.0|           -0.5|
|fill brim wham ba...|      9|  1.0|     0.25654763|
|hand controversia...|     10|  1.0|     0.19166666|
|highlight movie a...|     11|  1.0|            0.0|
|imgen company fun...|     12|  1.0|     0.13636364|
|performance under...|     13|  1.0|            0.0|
|fall hop joel sch...|     14|  1.0|           0.45|
|kelsey grammer go...|     15|  1.0|          

In [81]:
df2 = pos_label_df.withColumn("sentiment_score", sentiment_analysis_udf(pos_label_df['text'] ))
df2.show(20,True)

+--------------------+-------+-----+---------------+
|                text|pos_uid|label|sentiment_score|
+--------------------+-------+-----+---------------+
|film rat director...|      0|  1.0|         0.3125|
|   movie originality|      1|  1.0|            0.0|
|       anything good|      2|  1.0|            0.7|
|             letdown|      3|  1.0|            0.0|
|think joblo get l...|      4|  1.0|    -0.14107142|
|anything spectacu...|      5|  1.0|            0.6|
|attempt make char...|      6|  1.0|           0.15|
|yank us water pis...|      7|  1.0|            0.0|
|let begin say wor...|      8|  1.0|           -0.5|
|fill brim wham ba...|      9|  1.0|     0.25654763|
|hand controversia...|     10|  1.0|     0.19166666|
|highlight movie a...|     11|  1.0|            0.0|
|imgen company fun...|     12|  1.0|     0.13636364|
|performance under...|     13|  1.0|            0.0|
|fall hop joel sch...|     14|  1.0|           0.45|
|kelsey grammer go...|     15|  1.0|          

In [82]:
def condition(r):
    if (r >=0.1):
        label = "positive"
    elif(r <= -0.1):
        label = "negative"
    else:
        label = "neutral"
    return label

sentiment_udf = udf(lambda x: condition(x), StringType())